## Name: Tabular to graph with PAMs
### Date: 15/5/2024
### Status: Seems to work. Need more experimentation
### Idea: 
Following ideas on the discretization of features, I thought of calculating the similarity matrix of tabular data with their similarity based on their discretized bins.
So the steps are:
1. Feature Binning (using C4.5 i.e. a DT on each feature column separately, practically)
2. Triple generation (sample1- has_same_feature_bin_for_feature_1-sample2)
3. PAMs for fast creation of the NxN similarity matrix
   1. Interesting to check whether hihger-order could be useful?
   2. Other baseline to check the effect of PAMs
      1. Simply triples_df.groupby(['head', 'tail'])['rel].agg(list).apply(len) -> (will give (sample1, sample2)->num_of_similar_feauture_bins). It is the same as taking the NxNxR binary tensor and summing across R -> NxN
         1. We did that it is the **graph** model and it's the best one.
4. SVM on top of the pre-computed similarity matrix.



### Results:

1. Created **C45_Tr** as a feature transformer. It simply bins the features using a C4.5 strategy.
   1. Specifically, it does a label encoding of the features, transforming the feature value to the related ordinal bin number (e.g. 15.3->3), retaining the number of features in the dataset.
2. **generate_triples** is a simple helper function to create "edges" of the form "sample1 - has_same_feature_1 - sample2" across the dataset (where "has_same_feature_1" denotes their feature_1 value is in the same bin).
3. The next step is to pass them through PAMs for fast modelling creating an NxN matrix. We make it symmetric and then use this as the similarity matrix between them (higher value is more similar).
4. And an svm on-top

We experimented over 63 datasets, vs a DT/svm/knn on the original features and a DT/knn on the transformed features.

!The pam model heads-up is better to the DT!

Interestingly knn with transformed features is better than knn on the original features, but the same is not true for the DT (makes sense probably for DT).


In [ ]:
    model      rank
2   graph  3.193548
6  svm_TR  3.209677
3   pam_1  3.354839
0      DT  3.596774
1   DT_TR  4.000000
5     svm  4.677419
4   pam_2  5.967742

[[ 4 14  1  1]
 [ 2  9  1  1]
 [ 1 11  1  1]
 [ 1 10  1  1]
 [ 3 15  1  1]
 [ 6 18  1  1]
 [ 1 13  1  1]
 [ 3 13  1  1]
 [ 1  8  1  1]
 [ 2 10  1  1]]
(150, 4)
Output exceeds the size limit. Open the full output data in a text editor
[(0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 2, 6),
 (0, 2, 7),
 (0, 2, 8),
 (0, 2, 9),
 (0, 2, 10),
 (0, 2, 11),
 (0, 2, 12),
 (0, 2, 13),
 (0, 2, 14),
 (0, 2, 15),
 (0, 2, 16),
 (0, 2, 17),
 (0, 2, 18),
 (0, 2, 19),
 (0, 2, 20),
 (0, 2, 21),
 (0, 2, 22),
 (0, 2, 23),
 (0, 2, 24),
 (0, 2, 25),
...
 (28, 2, 31),
 (28, 2, 32),
 (28, 2, 33),
 (28, 2, 34),
 ...]
<150x150 sparse array of type '<class 'numpy.int64'>'
	with 3477 stored elements in Compressed Sparse Row format>
63
wdbc (1/64)
vote (2/64)
tokyo1 (3/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
tic_tac_toe (4/64)
threeOf9 (5/64)
spectf (6/64)
spect (7/64)
sonar (8/64)
saheart (9/64)
profb (10/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
prnn_synth (11/64)
prnn_crabs (12/64)
postoperative_patient_data (13/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
pima (14/64)
parity5 (15/64)
mux6 (16/64)
monk3 (17/64)
monk2 (18/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
monk1 (19/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
molecular_biology_promoters (20/64)
lupus (21/64)
labor (22/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
irish (23/64)
ionosphere (24/64)
hungarian (25/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
house_votes_84 (26/64)
horse_colic (27/64)
hepatitis (28/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
heart_statlog (29/64)
heart_h (30/64)
heart_c (31/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
haberman (32/64)
glass2 (33/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
german (34/64)
diabetes (35/64)
crx (36/64)
credit_g (37/64)
credit_a (38/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
corral (39/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
colic (40/64)
cleve (41/64)
bupa (42/64)
buggyCrx (43/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
breast_w (44/64)
breast_cancer_wisconsin (45/64)
breast_cancer (46/64)
breast (47/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
biomed (48/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
backache (49/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
australian (50/64)
appendicitis (51/64)
analcatdata_lawsuit (52/64)
analcatdata_japansolvent (53/64)
analcatdata_fraud (54/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
analcatdata_cyyoung9302 (55/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
analcatdata_cyyoung8092 (56/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
analcatdata_creditscore (57/64)
/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
analcatdata_boxing2 (58/64)
analcatdata_boxing1 (59/64)
analcatdata_bankruptcy (60/64)
analcatdata_asbestos (61/64)
analcatdata_aids (62/64)
    model      rank
2   graph  3.193548
6  svm_TR  3.209677
3   pam_1  3.354839
0      DT  3.596774
1   DT_TR  4.000000
5     svm  4.677419
4   pam_2  5.967742
/tmp/ipykernel_1710513/2773563811.py:159: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False)).reset_index(drop=True)
WINS
        graph  svm_TR  pam_1    DT  DT_TR   svm  pam_2
graph     0.0    28.0   35.0  39.0   40.0  40.0   54.0
svm_TR   32.0     0.0   31.0  36.0   37.0  44.0   51.0
pam_1    22.0    30.0    0.0  36.0   41.0  41.0   51.0
DT       22.0    25.0   25.0   0.0   26.0  42.0   54.0
DT_TR    21.0    24.0   20.0  23.0    0.0  40.0   54.0
svm      18.0     7.0   19.0  19.0   21.0   0.0   41.0
pam_2     8.0    10.0   11.0   8.0    8.0  19.0    0.0

In [ ]:
# Results (before adding relation importance ordering for pam, and without simple graph and svm_tr results.)

    model      rank
0      DT  2.951613
4     pam  2.967742
1   DT_TR  3.451613
3  kNN_TR  3.596774
2     kNN  3.903226
5     svm  4.129032

WINS (times the row model wins the column models. With 63 datasets, wins >= 63/2=32 denotes that the row model is better)
          DT  DT_TR   kNN  kNN_TR   svm   pam
DT       0.0   26.0  42.0    37.0  42.0  25.0
DT_TR   23.0    0.0  38.0    33.0  40.0  20.0
kNN     19.0   23.0   0.0    19.0  34.0  25.0
kNN_TR  24.0   28.0  34.0     0.0  35.0  28.0
svm     19.0   21.0  28.0    27.0   0.0  19.0
pam     36.0   41.0  36.0    34.0  41.0   0.0

In [1]:
from sklearn.datasets import load_breast_cancer, load_digits, load_iris
from pmlb import fetch_data

X, y = load_iris(return_X_y=True)
print(X.shape)

(150, 4)


In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import TransformerMixin
import numpy as np

class C45_Tr(TransformerMixin):
    def __init__(self, min_samples_leaf=4, max_leaf_nodes=126, random_state = 42):
        self.max_leaf_nodes = max_leaf_nodes
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.bins = []
        
    def fit(self, X, y):
        ## https://github.com/jyansir/tp-berta/blob/main/lib/feature_encoder.py
        for col in range(X.shape[1]):
            cur_X = X[:, col].reshape(-1,1)
            tree = DecisionTreeClassifier(min_samples_leaf=self.min_samples_leaf, max_leaf_nodes=self.max_leaf_nodes, \
                                          random_state=self.random_state).fit(cur_X, y).tree_
            tree_thresholds = []
            for node_id in range(tree.node_count):
                if tree.children_left[node_id] != tree.children_right[node_id]:
                    tree_thresholds.append(tree.threshold[node_id])
            tree_thresholds.append(cur_X.max())
            tree_thresholds.append(cur_X.min())
            self.bins.append(np.array(sorted(set(tree_thresholds))))
        return self
    
    def transform(self, X, y=None):
        tr = []
        for col in range(X.shape[1]):
            cur_tr = np.digitize(
                            X[:, col],
                            np.r_[-np.inf, self.bins[col][1:-1], np.inf],
                        ).astype(np.int64)
            tr.append(cur_tr)
        
        final = np.vstack(tr).T
        return final
tr = C45_Tr(min_samples_leaf=2)
X_tr = tr.fit_transform(X,y)
print(X_tr[:10,:])
print(X_tr.shape)    

[[ 4 14  1  1]
 [ 2  9  1  1]
 [ 1 11  1  1]
 [ 1 10  1  1]
 [ 3 15  1  1]
 [ 6 18  1  1]
 [ 1 13  1  1]
 [ 3 13  1  1]
 [ 1  8  1  1]
 [ 2 10  1  1]]
(150, 4)


In [75]:
from sklearn.feature_selection import f_classif

def generate_triples(X_tr, order_of_cols=[], order_of_indices=[], print_=False):
    
    " MAYBE WE COULD ALSO HAVE MORE DETAIL AS an edge of the form has_same_feature1_bin_1 instead of the current has_same_feature1" 
    from itertools import combinations
    triples = []
    if len(order_of_cols) == 0:
        order_of_cols = range(X_tr.shape[1])
    for col in order_of_cols:
        values =  X_tr[:, col]
        #print('val', values)
        for unq in np.unique(values):
            indices = np.argwhere(values == unq).flatten()
            #print(unq, indices.shape, indices)
            if len(order_of_indices) > 0:
                indices = [order_of_indices[index] for index in indices]
            if len(indices) > 1:
                for comb in combinations(indices, 2):
                    triples.append((comb[0], col, comb[1]))
    if print_:
        print(f'Transformed into {len(triples)}') 
    return triples

# Most important to the end
order_of_cols = np.argsort(f_classif(X, y)[0])[::-1]
triples = generate_triples(X_tr, order_of_cols)
# triples

In [15]:
from prime_adj.pam_creation import create_pam_matrices
import pandas as pd



triples_df = pd.DataFrame(triples, columns=['head', 'rel', 'tail'])
pam_lossless, pam_1, node2id, rel2id, broke_with_sparsity = create_pam_matrices(triples_df, max_order=1)
pam_lossless

<150x150 sparse array of type '<class 'numpy.int64'>'
	with 3477 stored elements in Compressed Sparse Row format>

In [71]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
import numpy as np
import pandas as pd
from sympy import re
from torch import rand
import cached_path
import time
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support



random_state = 42

number_of_cv_folds = 5

cv = StratifiedKFold(number_of_cv_folds, random_state=random_state, shuffle=True)

min_samples_leaf = 10
model_names = [
    
    
    "DT",
    "DT_TR",
    #"kNN",
    #"kNN_TR",
    'svm',
    'svm_TR',
    'pam_1',
    'pam_2',
    'graph',
 
    # "DT_Free",
    #"DiffProp",
    # "DiffProp_Free",
    # "PHCRegressor",
]


def set_seeds(seed=42):
    np.random.seed(seed)
    

set_seeds(random_state)

path_to_data_summary = "https://raw.githubusercontent.com/EpistasisLab/pmlb/master/pmlb/all_summary_stats.tsv"
dataset_df = pd.read_csv(cached_path.cached_path(path_to_data_summary), sep="\t")

classification_datasets = dataset_df[
    # (dataset_df["n_binary_features"] == dataset_df["n_features"])
    (dataset_df["task"] == "classification")
    & (dataset_df["n_classes"] == 2)
    & (dataset_df["n_features"] <= 100)
    & (dataset_df["n_instances"] <= 1000)
]["dataset"]

print(len(classification_datasets))

res = []
for dataset_index, classification_dataset in enumerate(classification_datasets[::-1][1:]):
    
    print(f"{classification_dataset} ({dataset_index + 1}/{len(classification_datasets) + 1})")
    X, y = fetch_data(classification_dataset, return_X_y=True)
    if y.max() != 1 or y.min() != 0:
        for wanted, actual in enumerate(np.unique(y)):
            y[y==actual] = wanted

    
        # train_X, test_X, train_y, test_y = train_test_split(
        #     X, y, stratify=y, test_size=0.2, random_state=random_state
        # )
    for model_name in model_names:
        #print(model_name)
        if "DT" in model_name:
            clf = DecisionTreeClassifier(
                random_state=random_state
            )
        elif 'kNN' in model_name:
            clf = KNeighborsClassifier(n_neighbors=3)
        elif 'svm' in model_name:
            clf = SVC()
        elif 'pam' in model_name:
            pass
        elif 'graph' in model_name:
            pass
        else:
            raise NotImplementedError
        # model = clf
        if 'TR' in model_name:
            model = Pipeline([
                ('tr', C45_Tr(min_samples_leaf=min_samples_leaf)),
                ('clf', clf)
            ])
        else:
            model = clf
        time_s = time.time()
        if 'pam' in model_name or 'graph' in model_name:
            y_true = []
            y_pred = []
            for train_indices, test_indices in cv.split(X,y):
                X_train, y_train = X[train_indices], y[train_indices]
                X_test, y_test = X[test_indices], y[test_indices]
                
                fi_clf = DecisionTreeClassifier(random_state=42)
                fi_clf.fit(X_train, y_train)
                order_of_cols = np.argsort(fi_clf.feature_importances_)[::-1] #np.arange(X.shape[1])#np.argsort(f_classif(X_train, y_train)[0])[::-1] 
                
                tr = C45_Tr(min_samples_leaf=min_samples_leaf)
                
                X_train = tr.fit_transform(X_train, y_train)
                X_test = tr.transform(X_test)
                
                
                
                  
                
                triples = generate_triples(np.vstack((X_train, X_test)), order_of_cols=order_of_cols, order_of_indices=np.concatenate((train_indices, test_indices)))

                triples_df = pd.DataFrame(triples, columns=['head', 'rel', 'tail'])
                if 'pam' in model_name:
                    pam_lossless, pam_1, node2id, rel2id, broke_with_sparsity = create_pam_matrices(triples_df, max_order=2, use_log=True)
                    hop = int(model_name.split('_')[1]) - 1
                    sim_matrix = pam_1[hop].todense()
                elif 'graph' in model_name:
                    from scipy.sparse import csr_array
                    counts_dict = triples_df.groupby(['head', 'tail'])['rel'].agg(list).apply(len).to_dict()
                    (rows, cols) = zip(*list(counts_dict.keys()))
                    vals = list(counts_dict.values())
                    sim_matrix = csr_array((vals, (rows, cols)), shape=(X.shape[0], X.shape[0])).todense()
                sim_matrix = np.maximum(sim_matrix, sim_matrix.T)
                clf = SVC(kernel='precomputed')
                clf.fit(sim_matrix[train_indices][:, train_indices], y_train)
                cur_pred = clf.predict(sim_matrix[test_indices][:, train_indices])
                y_pred.append(cur_pred)
                y_true.append(y_test)
            y_pred = np.concatenate(y_pred)
            y_true = np.concatenate(y_true)
            acc = accuracy_score(y_true, y_pred)
            (prec, rec, f1, sup) = precision_recall_fscore_support(
                y_true, y_pred, average="binary"
            )
        else:
            y_pred = cross_val_predict(model, X, y, cv=cv).astype(int)
            acc = accuracy_score(y, y_pred)
            (prec, rec, f1, sup) = precision_recall_fscore_support(
                y, y_pred, average="binary"
            )
        time_end = time.time() - time_s
        res.append((classification_dataset, model_name, time_end, acc, prec, rec, f1, sup))
        #print(res[-1])

res = pd.DataFrame(res, columns=['dataset', 'model', 'time', 'acc', 'pr', 'rec', 'f1', 'sup'])
# res.sort_values('f1', ascending=False)

# Step 2: Sort each group by 'f1'
sorted_df = res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False)).reset_index(drop=True)

# Step 3: Assign ranks within each group
sorted_df['rank'] = sorted_df.groupby('dataset').cumcount() + 1

# Step 4: Calculate mean rank for each model across all datasets
mean_ranks = sorted_df.groupby('model')['rank'].mean().reset_index().sort_values(by='rank')

print(mean_ranks)

63
wdbc (1/64)
vote (2/64)
tokyo1 (3/64)


/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


tic_tac_toe (4/64)
threeOf9 (5/64)
spectf (6/64)
spect (7/64)
sonar (8/64)
saheart (9/64)
profb (10/64)


/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


prnn_synth (11/64)
prnn_crabs (12/64)
postoperative_patient_data (13/64)


...

/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


analcatdata_creditscore (57/64)


/home/kbougatiotis/miniconda3/envs/prime/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


analcatdata_boxing2 (58/64)
analcatdata_boxing1 (59/64)
analcatdata_bankruptcy (60/64)
analcatdata_asbestos (61/64)
analcatdata_aids (62/64)
    model      rank
2   graph  3.193548
6  svm_TR  3.209677
3   pam_1  3.354839
0      DT  3.596774
1   DT_TR  4.000000
5     svm  4.677419
4   pam_2  5.967742


/tmp/ipykernel_1710513/2773563811.py:159: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False)).reset_index(drop=True)


In [72]:
print(mean_ranks)

    model      rank
2   graph  3.193548
6  svm_TR  3.209677
3   pam_1  3.354839
0      DT  3.596774
1   DT_TR  4.000000
5     svm  4.677419
4   pam_2  5.967742


In [74]:
#  res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False))
wins_score = np.zeros((len(model_names), len(model_names)))

for classification_dataset in res['dataset'].unique():
    cur_df = res[res['dataset'] == classification_dataset]
    # print(classification_dataset)
    # print(cur_df.sort_values('f1', ascending=False)[['model', 'time', 'acc', 'f1']])
    # print()
    cur_df = cur_df.set_index('model')
    score_metric = cur_df['f1']
    for i, m1 in enumerate(model_names):
        for j, m2 in enumerate(model_names[i:]):
            if cur_df.loc[m1]['f1'] > cur_df.loc[m2]['f1']:
                wins_score[i, j+i] += 1
            elif cur_df.loc[m1]['f1'] < cur_df.loc[m2]['f1']:
                wins_score[j+i, i] += 1
            else:
                pass
order_of_models = wins_score.mean(axis=1).argsort()[::-1]
wins_score = wins_score[order_of_models, :][:, order_of_models]
print('WINS')
print(pd.DataFrame(wins_score, columns = np.array(model_names)[order_of_models], index=np.array(model_names)[order_of_models]))

WINS
        graph  svm_TR  pam_1    DT  DT_TR   svm  pam_2
graph     0.0    28.0   35.0  39.0   40.0  40.0   54.0
svm_TR   32.0     0.0   31.0  36.0   37.0  44.0   51.0
pam_1    22.0    30.0    0.0  36.0   41.0  41.0   51.0
DT       22.0    25.0   25.0   0.0   26.0  42.0   54.0
DT_TR    21.0    24.0   20.0  23.0    0.0  40.0   54.0
svm      18.0     7.0   19.0  19.0   21.0   0.0   41.0
pam_2     8.0    10.0   11.0   8.0    8.0  19.0    0.0
